In [2]:
pwd

'/dccstor/bmfm-targets1/users/eyigoz/openai/notebooks/Final'

In [3]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"  # shows all output, not just the last one

In [4]:
import pandas as pd
import pickle
import os
import json
import re
import csv

In [5]:
import scipy.stats as stats

In [6]:
scifact = False
if scifact:
    data_dir = '/dccstor/bmfm-targets1/users/eyigoz/openai/notebooks/Final/scifact/'
    train_df = pd.read_json(data_dir+'claims_train.jsonl',lines=True)
    dev_df = pd.read_json(data_dir+'claims_dev.jsonl',lines=True)
    test_df = pd.read_json(data_dir+'claims_test.jsonl',lines=True)
    out_dir = '/dccstor/bmfm-targets1/users/eyigoz/openai/notebooks/Final/scifact_split_csvs/'
    def get_labels(evidence):
        labels = []
        for val in evidence.values():
            if len(val) > 0:
                for evi in val:
                    labels.append(evi['label'])
        labels = set(labels)
        if len(labels) > 0:
            return labels.pop()
            
        return None
            
    train_df['label'] = train_df.evidence.apply(lambda x: get_labels(x))
    
    train_df['label'] = train_df['label'].fillna('Not Enough Evidence')
    
    train_df.label.value_counts()
    eval_df = train_df


else:
    train_df = pd.read_json('averitec_data/train.json')
    dev_df = pd.read_json('averitec_data/dev.json')
    test_df = pd.read_json('averitec_data/test.json')
    # train_df = pd.read_csv('train_df.v1.csv')
    # dev_df = pd.read_csv('dev_df.v1.csv')
    # test_df = pd.read_csv('test_df.v1.csv')
    out_dir = '/dccstor/bmfm-targets1/users/eyigoz/openai/notebooks/Final/averitec_split_csvs/'
    eval_df = dev_df
    

In [7]:
eval_df.shape

(500, 14)

In [17]:
pkl_files = [x for x in os.listdir(out_dir) if x.endswith(".pkl") and "dev" in x ]
len(pkl_files)
# pkl_f = pkl_files[0] 
# pkl_f

242

In [18]:
summary_len = []
all_states = []
for pkl_f in pkl_files:
    states = pickle.load(open(out_dir+pkl_f,'rb'))
    all_states.append(states)
    summary_len.extend(states.apply(lambda x: len(x['summaries'])))
    

In [19]:
stats.describe(summary_len)

DescribeResult(nobs=484, minmax=(25, 565176), mean=58482.39462809917, variance=3027796487.9660997, skewness=4.431800308839087, kurtosis=27.53847616669141)

In [20]:
states = pd.concat(all_states)

In [21]:
states.sort_index(inplace=True)

In [22]:
states.head()

0    {'text': 'In a letter to Steve Jobs, Sean Conn...
1    {'text': 'Trump Administration claimed songwri...
2    {'text': 'Due to Imran Khan's criticism of Mac...
3    {'text': 'UNESCO declared Nadar community as t...
4    {'text': 'Republican Matt Gaetz was part of a ...
dtype: object

In [23]:

eval_df = pd.concat([eval_df.loc[states.index], states],axis=1)

In [24]:
eval_df.rename({0:'states'},axis=1,inplace=True)

In [25]:
eval_df.shape

(484, 15)

In [26]:
# def parse_results(raw_response,key):
#     clean_json = raw_response.strip('`').replace("json\n", "", 1).strip()
    
#     # Step 2: Parse to dict
#     try:
#         parsed = json.loads(clean_json)
#         return parsed[key]
#     except Exception as e:
#         print (e)
#         print (clean_json)
#     # print(parsed["result"])       # supports
#     # print(parsed["explanation"])  # full explanation

In [27]:
def parse_results(raw_response,keywords):
    clean_json = raw_response.strip('`').replace("json\n", "", 1).strip()
    cleaned_json_str = re.sub(r'\t', r'\\t', clean_json)
    
    # Step 2: Parse to dict
    try:
        parsed = json.loads(cleaned_json_str)
        keys = parsed.keys()
        for key in keys:
            all_in = True
            for keyword in keywords:
                if not keyword in key:
                    all_in = False
                
            if all_in:
                old_key = key
        #print ("--",old_key)        
        #parsed[replace_key] = parsed[old_key]  
        return parsed[old_key]
    except Exception as e:
        pass
        #print (e)
        #print (clean_json)
    # print(parsed["result"])       # supports
    # print(parsed["explanation"])  # full explanation

eval_df['evidence'] = eval_df.states.apply\
    (lambda x: parse_results(x['qa_pairs'],['question','answer']))

In [28]:
eval_df['search_query'] = eval_df.states.apply\
    (lambda x: x['search_query'])

In [29]:
eval_df['topic_questions'] = eval_df.states.apply\
    (lambda x: x['questions'])

In [30]:
eval_df.keys()

Index(['claim', 'required_reannotation', 'label', 'justification',
       'claim_date', 'speaker', 'original_claim_url', 'fact_checking_article',
       'reporting_source', 'location_ISO_code', 'claim_types',
       'fact_checking_strategies', 'questions', 'cached_original_claim_url',
       'states', 'evidence', 'search_query', 'topic_questions'],
      dtype='object')

In [ ]:
['claim', 'evidence', 'search_query', 'topic_questions']

In [53]:
for ind, row in eval_df.sample(1).iterrows():
    print("Claim:" , row['claim'])
    print("Label:", row['label'])

    print("Search query:", row['search_query'])
    print("Generated questons:\n", "\n".join(sorted(set(row['topic_questions']))))
    print("\n\n".join([str(qa) for qa in row['evidence']]))



Claim: The Trump RNC Acceptance Speech Was The First Time The White House Was Used For 'Purely Political Campaign Event'
Label: Refuted
Search query:  Trump RNC acceptance speech, White House political event, first time White House used for campaign, presidential campaign events at White House
Generated questons:
 1. Was the Trump RNC Acceptance Speech held at the White House?
2. Was the Trump RNC Acceptance Speech a purely political campaign event?
3. Had the White House been used for a purely political campaign event prior to the Trump RNC Acceptance Speech?
4. Was the Trump RNC Acceptance Speech the first time the White House was used for a purely political campaign event?
Here are the individual verifiable questions based on the claim:
{'question': 'Was the Trump RNC Acceptance Speech held at the White House?', 'answer': 'President Donald Trump delivered his acceptance speech for the Republican National Convention from the South Lawn of the White House on August 27, 2020.'}

{'ques

In [21]:
# eval_df['evidence'] = eval_df.states.apply\
#    (lambda x: parse_results(x['qa_pairs'],'question_answer_pairs'))

In [22]:
eval_df['evidence'].isnull().value_counts()

evidence
False    223
True       1
Name: count, dtype: int64

In [23]:
#eval_df.iloc[0].states['verifier_result']

In [24]:
eval_df['verifier_result_parsed'] = eval_df.states.apply\
    (lambda x: parse_results(x['verifier_result'],['result']))

In [25]:
#eval_df['verifier_result_parsed']

In [26]:
eval_df['verifier_result_parsed'] .isnull().value_counts()

verifier_result_parsed
False    224
Name: count, dtype: int64

In [27]:
eval_df['verifier_result_parsed'].value_counts()

verifier_result_parsed
refutes                 116
supports                 56
not_enough_evidence      44
conflicting_evidence      8
Name: count, dtype: int64

In [28]:
if scifact:
    label_map = {'not_enough_evidence':'Not Enough Evidence',\
             'Not Enough Evidence':'Not Enough Evidence',\
            'supports':'Supported',\
             'SUPPORT':'Supported',\
             'Supported':'Supported',\
             'CONTRADICT':'Refuted',\
            'refutes':'Refuted',\
             'Refuted':'Refuted',\
            'conflicting_evidence':'Not Enough Evidence',\
            'Conflicting Evidence/Cherrypicking':'Conflicting Evidence/Cherrypicking'}
else:
    label_map = {'not_enough_evidence':'Not Enough Evidence',\
             'Not Enough Evidence':'Not Enough Evidence',\
            'supports':'Supported',\
             'SUPPORT':'Supported',\
             'Supported':'Supported',\
             'CONTRADICT':'Refuted',\
            'refutes':'Refuted',\
             'Refuted':'Refuted',\
            'conflicting_evidence':'Conflicting Evidence/Cherrypicking',\
            'Conflicting Evidence/Cherrypicking':'Conflicting Evidence/Cherrypicking'}
    

In [29]:
eval_df.dropna(subset=['verifier_result_parsed'],inplace=True)

In [30]:
eval_df.columns

Index(['claim', 'required_reannotation', 'label', 'justification',
       'claim_date', 'speaker', 'original_claim_url', 'fact_checking_article',
       'reporting_source', 'location_ISO_code', 'claim_types',
       'fact_checking_strategies', 'questions', 'cached_original_claim_url',
       'states', 'evidence', 'search_query', 'topic_questions',
       'verifier_result_parsed'],
      dtype='object')

In [31]:
eval_df['orginal_label'] = eval_df.label

In [32]:
eval_df['pred_label'] = eval_df['verifier_result_parsed'].apply(lambda x: label_map[x])

In [33]:
eval_df['label'] = eval_df['label'].apply(lambda x: label_map[x])

In [34]:
eval_df.label.value_counts()

label
Refuted                               121
Supported                              67
Not Enough Evidence                    20
Conflicting Evidence/Cherrypicking     16
Name: count, dtype: int64

In [35]:
#eval_df['verifier_result_parsed'].value_counts()

In [36]:
eval_df['pred_label'].value_counts()

pred_label
Refuted                               116
Supported                              56
Not Enough Evidence                    44
Conflicting Evidence/Cherrypicking      8
Name: count, dtype: int64

In [37]:
#eval_df['evidence'].dropna().apply(lambda x: x[0].keys())

In [38]:
eval_df.head(2).T

,0,1
claim,"In a letter to Steve Jobs, Sean Connery refuse...",Trump Administration claimed songwriter Billie...
required_reannotation,False,False
label,Refuted,Refuted
justification,The answer and sources show that the claim was...,Seems that the Wzshington post accused the sin...
claim_date,31-10-2020,31-10-2020
speaker,None,None
original_claim_url,None,None
fact_checking_article,https://web.archive.org/web/20201130144023/htt...,https://web.archive.org/web/20201103001419/htt...
reporting_source,Facebook,Instagram
location_ISO_code,None,US


In [39]:
#eval_df.evidence

In [40]:
from sklearn.metrics import accuracy_score, f1_score

In [41]:
#accuracy_score(eval_df['pred_label'],eval_df.label)

"accuracy supported"
accuracy_score(eval_df['pred_label'].apply(lambda x: x == 'Supported'),\
               eval_df.label.apply(lambda x: x == 'Supported'))
"f1 supported"

f1_score(eval_df['pred_label'].apply(lambda x: x == 'Supported'),\
               eval_df.label.apply(lambda x: x == 'Supported'))
"accuracy refuted"
accuracy_score(eval_df['pred_label'].apply(lambda x: x == 'Refuted'),\
               eval_df.label.apply(lambda x: x == 'Refuted'))
"f1_refuted"
f1_score(eval_df['pred_label'].apply(lambda x: x == 'Refuted'),\
               eval_df.label.apply(lambda x: x == 'Refuted'))

"Accuracy not enough evidence"

accuracy_score(eval_df['pred_label'].apply(lambda x: x == 'Not Enough Evidence'),\
               eval_df.label.apply(lambda x: x == 'Not Enough Evidence'))

"f1 not enough evidencet"
f1_score(eval_df['pred_label'].apply(lambda x: x == 'Not Enough Evidence'),\
               eval_df.label.apply(lambda x: x == 'Not Enough Evidence'))

'accuracy supported'

0.7901785714285714

'f1 supported'

0.6178861788617886

'accuracy refuted'

...

0.7088607594936709

'Accuracy not enough evidence'

0.75

'f1 not enough evidencet'

0.125

In [42]:
eval_df.label.value_counts() / eval_df.shape[0]

label
Refuted                               0.540179
Supported                             0.299107
Not Enough Evidence                   0.089286
Conflicting Evidence/Cherrypicking    0.071429
Name: count, dtype: float64

In [43]:
eval_df.pred_label.value_counts() / eval_df.shape[0]

pred_label
Refuted                               0.517857
Supported                             0.250000
Not Enough Evidence                   0.196429
Conflicting Evidence/Cherrypicking    0.035714
Name: count, dtype: float64

# Create Submission csv

In [44]:
eval_df = eval_df[eval_df['evidence'].apply(lambda x: x != [])]

In [45]:
eval_df['evidence']

0      [{'question': 'Did Sean Connery write a letter...
1      [{'question': 'Were documents related to this ...
2      [{'question': 'Did Imran Khan criticize Macron...
4      [{'question': 'Is Matt Gaetz a Republican?', '...
5      [{'question': 'Do the articles published by th...
                             ...                        
221    [{'question': 'Did Nigeria gain independence?'...
222    [Does Nigeria currently grapple with multiple ...
223    [{'question': 'What are Nigeria's petrol price...
224    [{'question': 'Did Nigeria gain independence?'...
225    [{'question': 'Was there a drop in oil prices ...
Name: evidence, Length: 215, dtype: object

In [46]:
def fix_q_dict(x):
    if type(x[0]) is dict:
        return x
    elif type(x[0]) is str:
        res = []
        for qa in x:
            res.append({'question':qa.split("\t")[0],
             'answer':qa.split("\t")[-1]})
            
        return res

In [47]:
eval_df.dropna(subset=['evidence'],inplace=True)

In [48]:
eval_df['evidence'] = eval_df['evidence'].apply(lambda x: fix_q_dict(x))

In [56]:
new_samples = []
for i, sample in  eval_df.iterrows():
    claim = sample['claim']
    label = sample['pred_label']
    prediction_evidence = ""
    #print (len(sample['evidence']))
    for src_qa in sample['evidence']:    
        # print (sample['evidence'])
        prediction_evidence += src_qa["question"] + "\t\t\n" + src_qa["answer"] + "\t\t\n\n"
    #
    # print ("Claim:\n", claim)
    # print (prediction_evidence)
    new_samples.append([i, claim, prediction_evidence, label, 'pred'])

with open("leaderboard_submission/submission.csv".format("eyigoz"), mode="w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["id", "claim", "evi", "label", "split"])  # Write header
    writer.writerows(new_samples)

26

In [57]:
eval_df['claim_id'] = eval_df.index.values

In [58]:
#eval_df['claim_id']
eval_df.shape

(214, 22)

In [52]:
eval_df[['claim_id','claim','evidence','pred_label']].reset_index(drop=True).to_json('train_predictions.json', orient="records")

In [53]:
eval_df[['claim_id','claim','questions','claim_types','label']].reset_index(drop=True).to_json('train_gold.json', orient="records")

In [54]:
eval_df.iloc[0]['claim_types']

['Event/Property Claim']

In [55]:
# new_samples = []
# for i, sample in eval_df.iterrows():
#     claim = sample['claim']
#     label = sample['pred_label']
#     prediction_evidence = ""
#     for src_qa in sample['questions']:
#         prediction_evidence += src_qa["question"] + "\t\t\n" + src_qa["answer"] + "\t\t\n\n"
#     #
#     new_samples.append([i, claim, prediction_evidence, label, 'pred'])

# with open("leaderboard_submission/gold.csv".format("eyigoz"), mode="w", newline="") as file:
#     writer = csv.writer(file)
#     writer.writerow(["id", "claim", "evi", "label", "split"])  # Write header
#     writer.writerows(new_samples)